In [17]:
# Dependencies and Setup
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
# File to Load (Remember to Change These)


tomato_model_1 = "../Resources/movie_metadata_adjusted_revenue_starpower.csv"
# Read the Data
mv_df = pd.read_csv(tomato_model_1)
from scipy import stats
mv_df2 = mv_df[(np.abs(stats.zscore(mv_df['revenue'])) < 3)]
mv_df3 = mv_df2[(np.abs(stats.zscore(mv_df2['budget'])) < 3)]
tomato_df = mv_df3.dropna()
tomato_df.head()


,Column1,title,budget,popularity,release_date,revenue,runtime,vote_average,vote_count,star_power,first_genre
0,0,The Godfather,36703349.28,41.109264,1972,1.499126e+09,175,8.5,6024,1,Drama
1,1,The Shawshank Redemption,43134278.00,51.645403,1994,4.889955e+07,142,8.5,8358,1,Drama
2,2,Fight Club,96693277.31,63.869599,1999,1.547917e+08,139,8.3,9678,1,Comedy
3,3,Schindler's List,38930103.81,41.725123,1993,5.686725e+08,195,8.3,4436,1,Drama
4,4,One Flew Over the Cuckoo's Nest,14258364.31,35.529554,1975,5.179649e+08,133,8.3,3001,1,Classics


In [18]:
#Data selection for features
# compares star_power, budget with revenue as target.
X = tomato_df[["star_power", "budget"]]
# compares popularity, vote_average and vote_count with revenue as target
X_2 = tomato_df[["popularity", "vote_average", "vote_count"]]
y = tomato_df["revenue"].values.reshape(-1, 1)
print(X.shape, y.shape)

(1063, 2) (1063, 1)


In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)

In [20]:
X_scaler = MinMaxScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [22]:
# Create model and add layers  --- data 1
model = Sequential()
model.add(Dense(units=40, activation='relu', input_dim=2))
model.add(Dense(units=40, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [23]:
# Create model and add layers  --- data 2
model2 = Sequential()
model2.add(Dense(units=40, activation='relu', input_dim=3))
model2.add(Dense(units=40, activation='relu'))
model2.add(Dense(units=2, activation='softmax'))

In [24]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='msle',
              metrics=['accuracy'])

#reduce_mean no
#categorical_hinge yes
#
model2.compile(optimizer='adam',
              loss='msle',
              metrics=['accuracy'])
#
#adam

In [25]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 40)                120       
_________________________________________________________________
dense_7 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 82        
Total params: 1,842
Trainable params: 1,842
Non-trainable params: 0
_________________________________________________________________


In [26]:
model2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 40)                160       
_________________________________________________________________
dense_10 (Dense)             (None, 40)                1640      
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 82        
Total params: 1,882
Trainable params: 1,882
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.fit(
    X_train_scaled,
    y_train,
    epochs=60,
    shuffle=True,
    verbose=2
)

Train on 797 samples
Epoch 1/60
797/797 - 0s - loss: 357.8310 - acc: 0.5044
Epoch 2/60
797/797 - 0s - loss: 357.8308 - acc: 0.6223
Epoch 3/60
797/797 - 0s - loss: 357.8308 - acc: 0.4329
Epoch 4/60
797/797 - 0s - loss: 357.8308 - acc: 0.5295
Epoch 5/60
797/797 - 0s - loss: 357.8308 - acc: 0.5107
Epoch 6/60
797/797 - 0s - loss: 357.8308 - acc: 0.5056
Epoch 7/60
797/797 - 0s - loss: 357.8308 - acc: 0.5119
Epoch 8/60
797/797 - 0s - loss: 357.8308 - acc: 0.4981
Epoch 9/60
797/797 - 0s - loss: 357.8308 - acc: 0.5194
Epoch 10/60
797/797 - 0s - loss: 357.8308 - acc: 0.5458
Epoch 11/60
797/797 - 0s - loss: 357.8308 - acc: 0.5395
Epoch 12/60
797/797 - 0s - loss: 357.8308 - acc: 0.5772
Epoch 13/60
797/797 - 0s - loss: 357.8308 - acc: 0.5069
Epoch 14/60
797/797 - 0s - loss: 357.8308 - acc: 0.5207
Epoch 15/60
797/797 - 0s - loss: 357.8308 - acc: 0.5031
Epoch 16/60
797/797 - 0s - loss: 357.8308 - acc: 0.5270
Epoch 17/60
797/797 - 0s - loss: 357.8308 - acc: 0.4856
Epoch 18/60
797/797 - 0s - loss: 357

In [28]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

266/266 - 0s - loss: 355.5081 - acc: 0.1316
Normal Neural Network - Loss: 355.50811584013746, Accuracy: 0.1315789520740509


encoded_predictions = model.predict_classes(X_test_scaled[:5])
print(f"Predicted classes: {encoded_predictions}")

In [29]:
X_train2, X_test2, y_train, y_test = train_test_split(
    X_2, y, random_state=42)

In [30]:
X_scaler2 = MinMaxScaler().fit(X_train2)
X_train_scaled2 = X_scaler2.transform(X_train2)
X_test_scaled2 = X_scaler2.transform(X_test2)

In [31]:
model2.fit(
    X_train_scaled2,
    y_train,
    epochs=60,
    shuffle=True,
    verbose=2
)

Train on 797 samples
Epoch 1/60
797/797 - 1s - loss: 357.8313 - acc: 0.5157
Epoch 2/60
797/797 - 0s - loss: 357.8309 - acc: 0.4881
Epoch 3/60
797/797 - 0s - loss: 357.8309 - acc: 0.5169
Epoch 4/60
797/797 - 0s - loss: 357.8308 - acc: 0.4316
Epoch 5/60
797/797 - 0s - loss: 357.8308 - acc: 0.4843
Epoch 6/60
797/797 - 0s - loss: 357.8308 - acc: 0.4605
Epoch 7/60
797/797 - 0s - loss: 357.8308 - acc: 0.4617
Epoch 8/60
797/797 - 0s - loss: 357.8308 - acc: 0.4567
Epoch 9/60
797/797 - 0s - loss: 357.8308 - acc: 0.4868
Epoch 10/60
797/797 - 0s - loss: 357.8308 - acc: 0.4743
Epoch 11/60
797/797 - 0s - loss: 357.8308 - acc: 0.4630
Epoch 12/60
797/797 - 0s - loss: 357.8308 - acc: 0.4818
Epoch 13/60
797/797 - 0s - loss: 357.8308 - acc: 0.4693
Epoch 14/60
797/797 - 0s - loss: 357.8308 - acc: 0.4881
Epoch 15/60
797/797 - 0s - loss: 357.8308 - acc: 0.4693
Epoch 16/60
797/797 - 0s - loss: 357.8308 - acc: 0.4617
Epoch 17/60
797/797 - 0s - loss: 357.8308 - acc: 0.4730
Epoch 18/60
797/797 - 0s - loss: 357

In [32]:
model_loss2, model_accuracy2 = model2.evaluate(
    X_test_scaled2, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

266/266 - 0s - loss: 355.5081 - acc: 0.8233
Normal Neural Network - Loss: 355.50811584013746, Accuracy: 0.1315789520740509


#mape
1439/1439 - 0s - loss: 99.9999 - acc: 0.0674
Normal Neural Network - Loss: 99.99991681362043, Accuracy: 0.06740792095661163

#mae
1439/1439 - 0s - loss: 91092404.9590 - acc: 0.0028
Normal Neural Network - Loss: 91092404.9589993, Accuracy: 0.002779708243906498

#logcosh
1439/1439 - 0s - loss: 91092404.4253 - acc: 1.0000
Normal Neural Network - Loss: 91092404.42529534, Accuracy: 1.0

#categorical_hinge
1439/1439 - 0s - loss: 0.0000e+00 - acc: 0.0000e+00
Normal Neural Network - Loss: 0.0, Accuracy: 0.0

#msle
1439/1439 - 0s - loss: 272.5723 - acc: 0.9840
Normal Neural Network - Loss: 272.5723301805996, Accuracy: 0.9840166568756104